In [1]:
import tensorflow as tf 

2025-03-06 13:37:56.289281: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Centralized setting 

In [190]:
import numpy as np 
from sksurv.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split

# Load dataset 
X, y = load_breast_cancer()

# Unpack 
event, duration = zip(*y)
e = np.array(event)
t = np.array(duration)

# Drop columns that are not numerical 
X = X[X.columns[X.dtypes != "category"]]

# Train-test splitting 
train_idx, test_idx = train_test_split(
    np.arange(X.shape[0]), test_size=0.25, random_state=42, stratify=e.astype(int)
)
X_train, X_test = X.iloc[train_idx].to_numpy(), X.iloc[test_idx].to_numpy()
y_train, y_test = y[train_idx], y[test_idx]
e_train, e_test = e[train_idx], e[test_idx]
t_train, t_test = t[train_idx], t[test_idx]

X_train.shape, X_test.shape 

((148, 78), (50, 78))

In [191]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [192]:
x_train = np.log(t_train)
x_test = np.log(t_test)

knots = np.percentile(x, [0, 25, 50, 75, 100])
knots

array([4.82831374, 7.77267739, 8.38569183, 8.62443073, 9.11690843])

In [193]:
def spline_basis(x, knots, j):
    k_min = min(knots)
    k_max = max(knots)
    phi = (k_max - knots[j]) / (k_max - k_min)
    return np.maximum(0, x - knots[j]) ** 3 - phi * np.maximum(0, x - k_min) ** 3 - (1 - phi) * np.maximum(0, x - k_max) ** 3

S1_train = spline_basis(x_train, knots, j=1) 
S2_train = spline_basis(x_train, knots, j=2) 
S3_train = spline_basis(x_train, knots, j=3) 

# Spline design matrix 
D_train = np.transpose(np.vstack([np.ones_like(x_train), x_train, S1_train, S2_train, S3_train]))
D_train.shape 

(148, 5)

In [260]:
def spline_basis_derivative(x, knots, j):
    k_min = min(knots)
    k_max = max(knots)
    phi = (k_max - knots[j]) / (k_max - k_min)
    return 3 * np.maximum(0, x - knots[j]) ** 2 - 3 * phi * np.maximum(0, x - k_min) ** 2 - 3 * (1 - phi) * np.maximum(0, x - k_max) ** 2

dS1_train = spline_basis_derivative(x_train, knots, j=1) 
dS2_train = spline_basis_derivative(x_train, knots, j=2) 
dS3_train = spline_basis_derivative(x_train, knots, j=3) 

# Derivative spline design matrix 
dD_train = np.transpose(np.vstack([np.zeros_like(x_train), np.ones_like(x_train), dS1_train, dS2_train, dS3_train]))
dD_train.shape 

(148, 5)

In [267]:
beta = tf.Variable(tf.random.normal(shape=[1, int(X.shape[1])], mean=0, stddev=1, seed=42))
gamma = tf.Variable(tf.random.normal(shape=[1, int(Ds.shape[1])], mean=0, stddev=1, seed=42))

X_train_tf = tf.cast(X_train, dtype=tf.float32)
D_train_tf = tf.cast(D_train, dtype=tf.float32)
dD_train_tf = tf.cast(dD_train, dtype=tf.float32)

e_train_tf = tf.cast(e_train[:, None], dtype=tf.float32)
t_train_tf = tf.cast(t_train[:, None], dtype=tf.float32)

In [271]:
def loss():    
    eta = D_train_tf @ tf.transpose(gamma) + X_train_tf @ tf.transpose(beta)
 
    f1 = 1 / t_train_tf * dD_train_tf @ tf.transpose(gamma) * tf.math.exp(eta - tf.math.exp(eta))
    f2 = 1 / (1 + tf.math.exp(eta))

    return e_train_tf * f1 + (1 - e_train_tf) * f2
    

optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
for epoch in tf.range(10):

    with tf.GradientTape() as tape:
        loss_value = loss()
    
    # Compute gradients
    gradients = tape.gradient(loss_value, [gamma, beta])
    
    # Apply gradients to update gamma and beta
    optimizer.apply_gradients(zip(gradients, [gamma, beta]))
    
    print(f"Epoch {epoch}, Loss: {loss_value.numpy().mean()}")

Epoch 0, Loss: 0.5849466323852539
Epoch 1, Loss: 0.5039567947387695
Epoch 2, Loss: 0.4121197462081909
Epoch 3, Loss: 0.3399924635887146
Epoch 4, Loss: 0.25428447127342224
Epoch 5, Loss: 0.19022883474826813
Epoch 6, Loss: 0.12723389267921448
Epoch 7, Loss: 0.0855337381362915
Epoch 8, Loss: 0.05498019978404045
Epoch 9, Loss: 0.04202524572610855


# De-centralized setting 

In [ ]:
import numpy as np 
from sksurv.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split

# Load dataset 
X, y = load_breast_cancer()

# Unpack 
event, duration = zip(*y)
e = np.array(event)
t = np.array(duration)

# Drop columns that are not numerical 
X = X[X.columns[X.dtypes != "category"]]

# Train-test splitting 
train_idx, test_idx = train_test_split(
    np.arange(X.shape[0]), test_size=0.25, random_state=42, stratify=e.astype(int)
)
X_train, X_test = X.iloc[train_idx].to_numpy(), X.iloc[test_idx].to_numpy()
y_train, y_test = y[train_idx], y[test_idx]
e_train, e_test = e[train_idx], e[test_idx]
t_train, t_test = t[train_idx], t[test_idx]

X_train.shape, X_test.shape 